In [ ]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve


In [ ]:
spark = SparkSession.builder \
    .appName("assertion-status")\
    .master("local[2]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [ ]:
import time


embeddingsFile = 'PubMed-shuffle-win-2.bin'
embeddingsUrl = 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/PubMed-shuffle-win-2.bin'
# this may take a couple minutes
if not Path(embeddingsFile).is_file():
    urlretrieve(embeddingsUrl, embeddingsFile)

documentAssembler = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")\

assertion = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols(["document"])\
    .setOutputCol("assertion")\
    .setBefore(11)\
    .setAfter(13)\
    .setEmbeddingsSource(embeddingsFile,200,3)


finisher = Finisher() \
    .setInputCols(["assertion"]) \
    .setIncludeKeys(True)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    assertion,
    finisher
  ])


In [ ]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/negex.parquet"). \
        limit(3000)
data.cache()
data.count()
data.show()

In [ ]:
start = time.time()
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")
print (time.time() - start)

In [ ]:
result = model.transform(data)
result.select("sentence", "target", "finished_assertion").show()

In [ ]:
pipeline.write().overwrite().save("./assertion_pipeline")
model.write().overwrite().save("./assertion_model")

In [ ]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./assertion_pipeline")
sameModel = PipelineModel.read().load("./assertion_model")

In [ ]:
sameModel.transform(data).select("sentence", "target", "finished_assertion").show()